# Perform several scaffold analyses on generated molecules conditioned on profiles from selected genes and DMSO
- Quantify common scaffolds between generated molecules and agonists from the ExCAPE Database:  
`common_MurkoScafold_quantification_Gene_vs_Excape_mean.csv'`
- Perform scaffold enrichment analysis of generated molecules conditioned on each gene vs DMSO, and select large scaffolds consistently enriched in 3 repetitions:   
`LargeConsistentlyEnrichedScafolds.pkl`
- Collect ExCAPE molecules containing enriched scaffolds (ExCAPE hits)
`Excape_agonists_with_LargeConsistentlyEnrichedScafolds`
- Collect generated molecules with enriched scaffolds and high similarity to ExCAPE hits  
`GeneratedMols_with_LargeConsistentlyEnrichedScafolds_and_HighExcapeSim__...`


In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 50)
from joblib import Parallel, delayed
from tqdm import tqdm
import pickle
import scipy.stats as stats
from rdkit.Chem import PandasTools
from IPython.display import display, HTML

import logging 
logging.basicConfig(level=logging.INFO, format ='%(levelname)s - %(message)s')

import cpmolgan.utils as utils
import cpmolgan.nearest_neighbors as nn


### Arguments

In [2]:
args = {
    'p-value': 0.01, 
    'ecfp_sim_thresh':0.6,
    'filename_data_Excape':'../../data/ExCAPE_ligands_agonist_noTrainSet.csv',
    'results_dir': 'results',
    'generated_ref_dir':"REP_Valid_PassPhysChemFilter/generated_mols",
    'generated_ref_filename':"GENE__20000_Valid_PassPhysChemFilter.csv",
    'repetitions': ["n1","n2","n3"],
    'Excape_genes':["TP53","BRCA1","NFKB1","HSPA5", "CREBBP", "STAT1", "STAT3","HIF1A", "NFKBIA","JUN","PRKAA1","PDPK1"],
                  #   0      1      2        3        4         5        6       7        8       9      10      11 
}

output_dir = os.path.join( args['results_dir'], 'scafold_analysis')


if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## 1. Read Excape data and compute scafolds
Reference data to compare against

In [3]:
data_Excape = pd.read_csv(args['filename_data_Excape'],index_col=0)
data_Excape = data_Excape[ ['Gene_Symbol','SMILES_standard'] ].drop_duplicates()
data_Excape = data_Excape.loc[ data_Excape.Gene_Symbol.isin(args["Excape_genes"])].reset_index(drop=True)

# Add columns to warn compounds with multiple targets
def merge_targets(x):
    targets = x.Gene_Symbol.values.astype(str)
    if len(targets)>1:
        return ";".join(targets)
    else: 
        return None   
multiple_targets = data_Excape.groupby(by='SMILES_standard').apply(lambda x:  merge_targets(x) )
multiple_targets = pd.DataFrame(multiple_targets, columns=['multiple_targets']).reset_index()
data_Excape = data_Excape.merge(multiple_targets, on='SMILES_standard', how='left')

data_Excape['murcko_scafold'] = Parallel(n_jobs=16)(delayed(utils.generate_scaffold)(smiles, generic=False) for smiles in data_Excape.SMILES_standard )


## 2. Read generated data and compute scafolds

In [4]:
data_gen_dict = dict()

for r, rep in enumerate(args['repetitions']):
    logging.info('repetition %s'%rep)
    data_gen_dict[r] = pd.DataFrame()
    
    for gene in tqdm(['DMSO']+ args['Excape_genes']):
        
        generated_file = os.path.join( args['results_dir'], args["generated_ref_dir"], args["generated_ref_filename"].replace("GENE", gene) ).replace("REP",rep) 
        temp = pd.read_csv( generated_file, index_col=0 )
        
        # Compute scafolds if needed and save data with scafolds 
        if not( 'murcko_scafold' in temp.columns):
            temp['murcko_scafold'] = Parallel(n_jobs=16)(delayed(utils.generate_scaffold)(smiles, False) for smiles in temp.SMILES_standard )
            temp['murcko_scafold_generic'] = Parallel(n_jobs=16)(delayed(utils.generate_scaffold)(smiles, True) for smiles in temp.SMILES_standard )
            temp.to_csv( generated_file )
        data_gen_dict[r] = pd.concat( [ data_gen_dict[r], temp ] )
    data_gen_dict[r] = data_gen_dict[r].reset_index(drop=True)
    
data_gen_df = pd.concat( data_gen_dict ).reset_index().drop(columns='level_1').rename(columns={'level_0':'repetition'})

INFO - repetition n1
100%|██████████| 13/13 [00:01<00:00,  6.92it/s]
INFO - repetition n2
100%|██████████| 13/13 [00:01<00:00,  8.16it/s]
INFO - repetition n3
100%|██████████| 13/13 [00:01<00:00,  8.24it/s]


## 3. Global common scafold quantification between generated and Excape
Quantify common scafolds for each gene between generated and Excape agonist

In [5]:
def quantify_common_scafolds(df_1, df_2, scafold_col, smiles_col='SMILES_standard'):
    
    unique_scaf_1 =  df_1[scafold_col].unique()
    unique_scaf_2 =  df_2[scafold_col].unique()
    
    Nscaf_1 = len(unique_scaf_1)
    common_idx_1 = df_1[scafold_col].isin( unique_scaf_2 )
    Ncommon_scaf_1 = len( df_1.loc[common_idx_1, scafold_col].unique() )
    N_comm_scaf_cpds_1 =  len( df_1.loc[common_idx_1, smiles_col].unique() )
    
    Nscaf_2 = len(unique_scaf_2)
    common_idx_2 = df_2[scafold_col].isin( unique_scaf_1 )
    Ncommon_scaf_2 = len( df_2.loc[common_idx_2, scafold_col].unique() )
    N_comm_scaf_cpds_2 =  len( df_2.loc[common_idx_2, smiles_col].unique() )
    
    assert Ncommon_scaf_1==Ncommon_scaf_2
    
    # convert to float so that pandas includes all columns when averaging
    Nscaf_1, Nscaf_2, Ncommon_scaf_1, N_comm_scaf_cpds_1, N_comm_scaf_cpds_2 =  float(Nscaf_1), float(Nscaf_2), float(Ncommon_scaf_1), float(N_comm_scaf_cpds_1), float(N_comm_scaf_cpds_2)

    return  Nscaf_1, Nscaf_2, Ncommon_scaf_1, N_comm_scaf_cpds_1, N_comm_scaf_cpds_2


In [6]:
output_filename = os.path.join( output_dir, 'common_MurkoScafold_quantification_Gene_vs_Excape_mean.csv')
scafold_col = 'murcko_scafold'

if not os.path.isfile(output_filename):
    
    murkos = dict()    
    for r in range(len(args['repetitions'])):
        logging.info('repetition %s'%(r+1))
        
        murkos[r] = pd.DataFrame(columns=['Gene_Symbol_Excape','Gene_Symbol_generated',
                                   'N_unique_cpds_Excape','N_unique_cpds_generated',
                                   'N_scaf_Excape', 'N_scaf_generated',
                                   'N_common_scaf', 'N_cpds_common_scaf_Excape','N_cpds_common_scaf_generated'])

        for gene in tqdm(args['Excape_genes']):
            
            # Select data for current genes
            df_excape = data_Excape.loc[ data_Excape.Gene_Symbol == gene, ['SMILES_standard',scafold_col] ] 
            df_gen = data_gen_dict[r].loc[ data_gen_dict[r].Gene_Symbol == gene, ['SMILES_standard', scafold_col] ]

            # Quantify common scafolds
            NExcape, Ngen, Ncomm, Ncpds_Excape, Ncpds_gen = quantify_common_scafolds( df_excape, df_gen, scafold_col)
            murkos[r].loc[len(murkos[r])] = [ gene, gene, 
                                             len(df_excape.SMILES_standard.unique() ), len(df_gen.SMILES_standard.unique()), 
                                             NExcape, Ngen, 
                                             Ncomm, Ncpds_Excape, Ncpds_gen ]

    # Mean among repetitions    
    murkos = pd.concat( murkos ).reset_index().rename(columns={'level_0':'repetition'}).drop(columns='level_1')
    group_cols = ['Gene_Symbol_Excape','Gene_Symbol_generated','N_unique_cpds_Excape','N_scaf_Excape']
    means = murkos.groupby(by=group_cols).mean().drop(columns='repetition').reset_index()
    stds = murkos.groupby(by=group_cols).std().drop(columns='repetition').reset_index()
    murkos = means.merge(stds, on=group_cols, suffixes=['_mean','_std'])
    
    # Save results
    murkos.to_csv(output_filename)    
else:
    logging.info('File already exists %s'%output_filename)


INFO - File already exists results/scafold_analysis/common_MurkoScafold_quantification_Gene_vs_Excape_mean.csv


## 4. Scafold Enrichment analysis for each gene vs DMSO

#### 4.1 perform enrichment analysis

In [7]:
directionality = ['enriched','reduced'][0]

scaf_col = 'murcko_scafold'
output_filename = os.path.join( output_dir, 'Significantly'+directionality.capitalize()+'Scafolds_per_gene_vs_DMSO__pValue'+str(args['p-value'])+'.pkl')

if not os.path.isfile(output_filename):
    
    significant_scafolds = dict()
    signifcant_p_values = dict()

    for r in range(len(args['repetitions'])):
        
        significant_scafolds[r]= dict()
        signifcant_p_values[r] = dict()

        rep_data = data_gen_df.loc[ data_gen_df.repetition==r]
        counts = rep_data.groupby( by= ['Gene_Symbol','murcko_scafold']).apply(lambda x: len(x.SMILES_standard.unique()))
        counts = counts.reset_index().rename(columns={0:"Unique_SMILES"})

        for gene in  args['Excape_genes']:

            gene_df = counts.loc[ counts.Gene_Symbol==gene]
            dmso_df = counts.loc[ counts.Gene_Symbol=='DMSO']  
            significant_scafolds[r][gene] = []
            signifcant_p_values[r][gene] = []
            selected_scafolds = gene_df.loc[ gene_df.Unique_SMILES>5, scaf_col].unique() # to reduce time skipping useless scafolds apperaring few times

            for scafold in tqdm(selected_scafolds):

                # Contingency matrix
                cont_mat = pd.DataFrame( index=['scafold_present','scafold_absent'], columns=['cond_gene','cond_dmso'])
                cont_mat.loc['scafold_present','cond_gene'] = gene_df.loc[ gene_df[scaf_col]==scafold, 'Unique_SMILES'].sum()  # actually only one element
                cont_mat.loc['scafold_absent','cond_gene'] = gene_df.loc[ gene_df[scaf_col]!=scafold, 'Unique_SMILES'].sum()
                cont_mat.loc['scafold_present','cond_dmso'] =  dmso_df.loc[ dmso_df[scaf_col]==scafold, 'Unique_SMILES'].sum()
                cont_mat.loc['scafold_absent','cond_dmso'] = dmso_df.loc[ dmso_df[scaf_col]!=scafold, 'Unique_SMILES'].sum()
                
                # Fisher test
                _, p = stats.fisher_exact(cont_mat)
                significant = (p<args['p-value'])
                gene_proportion = cont_mat.loc['scafold_present','cond_gene']/cont_mat.cond_gene.sum()
                dmso_proportion  = cont_mat.loc['scafold_present','cond_dmso']/cont_mat.cond_dmso.sum()
                
                if directionality=='enriched':
                    significantly_changed = significant & (gene_proportion >dmso_proportion)
                elif directionality=='reduced':
                    significantly_changed = significant & (gene_proportion <dmso_proportion)
                    
                if significantly_changed:
                    significant_scafolds[r][gene].append(scafold)
                    signifcant_p_values[r][gene].append(str(p))

            logging.info("rep %i - %s Significant Scafolds: %i"%(r, gene,len(significant_scafolds[r][gene])))
    
    # Select scafolds which are consitently enriched in all repetitions
    consistent_scafolds = dict()
    for gene in significant_scafolds[0].keys():
        for r in significant_scafolds.keys():
            if r ==0:
                common_all_reps = set(significant_scafolds[r][gene])
            else:
                common_all_reps = common_all_reps.intersection( significant_scafolds[r][gene])
        consistent_scafolds[gene] = list(common_all_reps)

    # Save Results 
    scafolds_dict = {'significant_scafolds':significant_scafolds,
                 'signifcant_p_values':signifcant_p_values,
                 'consistent_scafolds':consistent_scafolds}
    pickle.dump(scafolds_dict,  open(output_filename, "wb"))

else:
    logging.info("File already exists. Loading it \n%s"%output_filename)
    scafolds_dict = pickle.load(open(output_filename, "rb"))
    consistent_scafolds = scafolds_dict['consistent_scafolds']

INFO - File already exists. Loading it 
results/scafold_analysis/SignificantlyEnrichedScafolds_per_gene_vs_DMSO__pValue0.01.pkl


#### 4.2 Select large scafolds consistently enriched in all repetitions

In [8]:
output_filename = os.path.join( output_dir, 'LargeConsistentlyEnrichedScafolds.pkl')

if not os.path.isfile(output_filename):
    min_scafold_size = 15
    large_consistent_scaf = dict()
    for gene in consistent_scafolds.keys():
        large_consistent_scaf[gene] = [ s for s in consistent_scafolds[gene] if utils.num_atoms_from_smiles(s)>=min_scafold_size ]
    pickle.dump(large_consistent_scaf,  open(output_filename, "wb"))
else:
    logging.info("File already exists. Loading it \n%s"%output_filename)
    large_consistent_scaf = pickle.load(open(output_filename, "rb"))


INFO - File already exists. Loading it 
results/scafold_analysis/LargeConsistentlyEnrichedScafolds.pkl


## 5. Find ExCAPE agonists containing large-consistently-enriched scafolds

In [9]:
output_filename = os.path.join( output_dir, 'Excape_agonists_with_LargeConsistentlyEnrichedScafolds.csv')
if not os.path.isfile(output_filename):
    Excape_enriched_scafs = pd.DataFrame()
    for gene in args['Excape_genes']:
        commom_excape_idx = data_Excape['murcko_scafold'].isin(large_consistent_scaf[gene]) & (data_Excape.Gene_Symbol==gene) 
        commom_excape = data_Excape.loc[commom_excape_idx].sort_values(by='murcko_scafold').reset_index(drop=True)
        commom_excape = commom_excape.rename(columns={'SMILES_standard':'SMILES_standard_Excape'})
        commom_excape['Gene_Symbol']=gene
        commom_excape['Excape_hit_name']=[ gene+'_#'+str(i+1) for i in range(len(commom_excape))]
        Excape_enriched_scafs = pd.concat([Excape_enriched_scafs,commom_excape])
    Excape_enriched_scafs = Excape_enriched_scafs.reset_index(drop=True)
    Excape_enriched_scafs.to_csv(output_filename)
else: 
    logging.info('File already exists \n%s'%output_filename)
    Excape_enriched_scafs = pd.read_csv(output_filename,index_col=0)

INFO - File already exists 
results/scafold_analysis/Excape_agonists_with_LargeConsistentlyEnrichedScafolds.csv


## 6. Find generated molecules containing large-consistently-enriched scafolds and with high simmilairty to the corresponding agonist


In [10]:
for plot_gene in args['Excape_genes']:

    output_file = os.path.join(output_dir,'GeneratedMols_with_LargeConsistentlyEnrichedScafolds_and_HighExcapeSim__'+plot_gene+'.csv')
    logging.info('\n-- Gene %s'%(plot_gene))
    generated_hits = dict()
    scafolds_in_agonists = []
    scafolds_with_hits = []
    output_hits_df = pd.DataFrame()

    for scafold in large_consistent_scaf[plot_gene]:

        excape_agonists = data_Excape.loc[ (data_Excape[scaf_col]==scafold) & (data_Excape.Gene_Symbol==plot_gene) ]
        if len(excape_agonists)>0:
            scafolds_in_agonists.append(scafold)

            for Excape_smiles in excape_agonists.SMILES_standard.values:

                Excape_ecfp = nn.mol_to_ecfp( nn.smile_to_mol (Excape_smiles)) 
                generated_mols = data_gen_df.loc[ (data_gen_df.Gene_Symbol==plot_gene) & (data_gen_df[scaf_col]==scafold) ]
                generated_mols = pd.DataFrame( generated_mols.SMILES_standard.value_counts().reset_index() ).rename(columns={'index':"SMILES_standard",'SMILES_standard':'counts'})
                generated_mols['Murcko'] = scafold
                generated_mols['Excape_SMILES_standard'] = Excape_smiles
                generated_mols["ecfp"]  = nn.smiles_to_ecfps_parallel( generated_mols.SMILES_standard.values )
                generated_mols['sim2Excape'] = nn.bulk_dice_sim((Excape_ecfp, generated_mols.ecfp.values))
                generated_mols = generated_mols.sort_values(by='sim2Excape', ascending=False)
                generated_mols = generated_mols.loc[ generated_mols.sim2Excape>args['ecfp_sim_thresh']].reset_index(drop=True)
                output_hits_df = pd.concat([output_hits_df,generated_mols])

                if len(generated_mols)>0:
                    scafolds_with_hits.append(scafold)
                    generated_hits[Excape_smiles] = generated_mols.SMILES_standard.values

    # Sumarize results 
    scafolds_with_hits = np.unique(scafolds_with_hits)
    logging.info('%s Large-consistently-enriched scafolds: %i'%(plot_gene,len(large_consistent_scaf[plot_gene])))
    logging.info('of which are found in Excape agonists: %i'%(len(scafolds_in_agonists)))
    logging.info('of which have generated molecules closed to an Excape agonist: %i'%(len(scafolds_with_hits)))
    logging.info('Num Excape agonists with highly simmilar genrated molecules: %i'%(len(generated_hits)))

    # Save results 
    output_hits_df['GeneSymbol']=plot_gene
    if len(output_hits_df)>0:
        output_hits_df = output_hits_df.drop(columns='ecfp').reset_index(drop=True)
        if not os.path.isfile(output_file): 
            output_hits_df.to_csv(output_file)
        else: 
            logging.info('File already exists')
    

INFO - 
-- Gene TP53
INFO - TP53 Large-consistently-enriched scafolds: 3
INFO - of which are found in Excape agonists: 0
INFO - of which have generated molecules closed to an Excape agonist: 0
INFO - Num Excape agonists with highly simmilar genrated molecules: 0
INFO - 
-- Gene BRCA1
INFO - BRCA1 Large-consistently-enriched scafolds: 8
INFO - of which are found in Excape agonists: 1
INFO - of which have generated molecules closed to an Excape agonist: 1
INFO - Num Excape agonists with highly simmilar genrated molecules: 1
INFO - File already exists
INFO - 
-- Gene NFKB1
INFO - NFKB1 Large-consistently-enriched scafolds: 25
INFO - of which are found in Excape agonists: 2
INFO - of which have generated molecules closed to an Excape agonist: 1
INFO - Num Excape agonists with highly simmilar genrated molecules: 1
INFO - File already exists
INFO - 
-- Gene HSPA5
INFO - HSPA5 Large-consistently-enriched scafolds: 18
INFO - of which are found in Excape agonists: 0
INFO - of which have generat